In [1]:
#Cargamos las libreria que vamos a utilizar
import numpy as np
import nltk
import pandas as pd 
import ast
import matplotlib as plt
from textblob import TextBlob
import pyarrow
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
#cargamos el archivo json
with open("C:\\Users\\Dr-Pc\\OneDrive\\Escritorio\\PI MLOps - STEAM\\australian_users_items.json", 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]


In [3]:
#convertimos en un dataframe el archivo y lo desanidamos completamente
user_items = pd.DataFrame(data)
user_items_explored = user_items.explode('items')
user_items_normalized = pd.json_normalize(user_items_explored['items'])
user_items_explored.reset_index(drop=True, inplace=True)
user_items_normalized.reset_index(drop=True, inplace=True)
user_items_final = pd.concat([user_items_explored.drop('items',axis=1), user_items_normalized], axis=1)

In [4]:
#eliminamos las filas en donde todas las columnas sean nulas 
user_items_final.dropna(how="all",inplace=True)

In [5]:
#aquí renombramos las columnas para que sea vean más prolijas y esteticas
user_items_final.rename(columns={'user_id': 'User ID'}, inplace=True)
user_items_final.rename(columns={'items_count': 'Items Count'}, inplace=True)
user_items_final.rename(columns={'steam_id': 'Steam ID'}, inplace=True)
user_items_final.rename(columns={'user_url': 'User URL'}, inplace=True)
user_items_final.rename(columns={'item_id': 'Item ID'}, inplace=True)
user_items_final.rename(columns={'item_name': 'Item Name'}, inplace=True)
user_items_final.rename(columns={'playtime_forever': 'Play Time Forever'}, inplace=True)
user_items_final.rename(columns={'playtime_2weeks': 'PlayTime 2Weeks'}, inplace=True)

In [6]:
# Llenar valores nulos en la columna playtime_forever con 0.0
user_items_final["PlayTime 2Weeks"].fillna(0.0, inplace=True)
user_items_final["Play Time Forever"].fillna(0.0, inplace=True)

In [7]:
#Eliminamos datos nulos
user_items_final.dropna(subset=['Item ID'], inplace=True)
user_items_final.dropna(subset=['Item Name'], inplace=True)

In [19]:
#vemos como van quedando los datos y la cantidad
 user_items_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   User ID            object 
 1   Item ID            object 
 2   Play Time Forever  float64
dtypes: float64(1), object(2)
memory usage: 116.6+ MB


In [10]:
# Función para convertir valores a float si no son nulos
def convertir_a_float(valor):
    try:
        # Intenta convertir a float si no es nulo
        return float(valor)
    except (ValueError, TypeError):
        # Si es nulo, devuelve el mismo valor sin cambios
        return valor

# Aplica la función a las columnas 'Play Time Forever' y 'PlayTime 2Weeks'
user_items_final['Play Time Forever'] = user_items_final['Play Time Forever'].apply(convertir_a_float)
user_items_final['PlayTime 2Weeks'] = user_items_final['PlayTime 2Weeks'].apply(convertir_a_float)


In [11]:
#eliminamos datos duplicados que hayan en el dataframe
user_items_final.drop_duplicates(inplace=True)

In [12]:
#reiniciamos el index
user_items_final.reset_index(drop=True, inplace=True)

In [13]:
#eliminamos las columnas
user_items_final.drop('PlayTime 2Weeks' , axis = 1, inplace = True)
user_items_final.drop('User URL' , axis = 1, inplace = True)
user_items_final.drop('Steam ID' , axis = 1, inplace = True)
user_items_final.drop('Items Count' , axis = 1, inplace = True)
user_items_final.drop('Item Name' , axis = 1, inplace = True)

In [20]:
#vemos como van quedando los datos y las columnas
user_items_final.head(20)

,User ID,Item ID,Play Time Forever
0,76561197970982479,10,6.0
1,76561197970982479,20,0.0
2,76561197970982479,30,7.0
3,76561197970982479,40,0.0
4,76561197970982479,50,0.0
5,76561197970982479,60,0.0
6,76561197970982479,70,0.0
7,76561197970982479,130,0.0
8,76561197970982479,300,4733.0
9,76561197970982479,240,1853.0


In [17]:
#borramos las filas en donde el valor es 0
df_user_items = user_items_final[user_items_final["Play Time Forever"] != 0]
df_user_items

,User ID,Item ID,Play Time Forever
0,76561197970982479,10,6.0
2,76561197970982479,30,7.0
8,76561197970982479,300,4733.0
9,76561197970982479,240,1853.0
10,76561197970982479,3830,333.0
...,...,...,...
5094098,76561198329548331,304930,677.0
5094099,76561198329548331,227940,43.0
5094102,76561198329548331,388490,3.0
5094103,76561198329548331,521570,4.0


In [37]:
#vemos como van quedando los datos y la cantidad 
user_items_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19899 entries, 255 to 5093995
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   User ID            19899 non-null  object 
 1   Item ID            19899 non-null  object 
 2   Play Time Forever  19899 non-null  float64
dtypes: float64(1), object(2)
memory usage: 621.8+ KB


In [22]:
#convertimos el archivo a parquet
user_items_final.to_parquet("user_items_final.parquet")